## Understanding Sentiment Analysis and RNNs
### Sentiment Analysis
Definition:

Sentiment analysis is a technique used to determine the emotional tone behind a piece of text. It identifies whether the sentiment is positive, negative, or neutral.

#### Applications:

- Customer Feedback: Understanding customer opinions about products or services.
- Social Media Monitoring: Gauging public sentiment on social media platforms.
- Market Research: Analyzing consumer preferences and trends.
- Political Analysis: Assessing public opinion on political matters.
- Healthcare: Evaluating patient feedback to improve services.

#### How RNNs Differ from Traditional Feedforward Neural Networks
1) Feedforward Neural Networks (FNNs):
- Data flows in one direction from input to output.
- Suitable for tasks with independent input data, like image classification.
2) Recurrent Neural Networks (RNNs):
- Data flows in a loop, allowing the network to retain memory of previous inputs.
- Ideal for sequential data where inputs are dependent on previous ones, like language processing.

#### Hidden States and Information Passing in RNNs
1) Hidden States: Vectors that store information about previous inputs, acting as the network's memory.
2) Information Passing:
- Initialization: Starts with a zero vector or a learned parameter.
- Update: At each time step, the hidden state is updated based on the current input and the previous hidden state, enabling the network to capture temporal dependencies.

#### Common Issues with RNNs: Vanishing and Exploding Gradients
1) Vanishing Gradients:
- Problem: Gradients become very small, hindering the network's ability to learn long-term dependencies.
- Solution: Use architectures like LSTM or GRU that manage gradients better.

2) Exploding Gradients:
- Problem: Gradients become too large, causing instability.
- Solution: Apply gradient clipping to limit gradient values.

By addressing these issues, RNNs can effectively learn from sequential data, making them powerful for tasks like sentiment analysis and language modeling.

In [66]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

# Parameters
max_features = 10000  # Top most frequent words to consider
maxlen = 500  # Max sequence length
batch_size = 32
embedding_dims = 32
epochs = 10

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [67]:
# Pad sequences
print('Pad sequences (samples x time)')
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 500)
x_test shape: (25000, 500)


In [68]:
# Define the model architecture
model = Sequential()
model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

c:\Users\annma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [71]:
print(f"x_train shape: {len(x_train)}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {len(x_test)}")
print(f"y_test shape: {y_test.shape}")

x_train shape: 25000
y_train shape: (25000,)
x_test shape: 25000
y_test shape: (25000,)


In [72]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2,
                    callbacks=[early_stopping])

# Monitor the training process
print(history.history.keys())

Epoch 1/10
186/625 ━━━━━━━━━━━━━━━━━━━━ 8:34 1s/step - accuracy: 0.6071 - loss: 0.6344

In [ ]:
# Evaluate the model
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)